# Úkol č. 2 - předzpracování dat a binární klasifikace (do 10. listopadu)

  * Cílem thoto úkolu je vyzkoušet si naučit prediktivní model pro binární klasifikaci.
  * Budete se muset vypořádat s příznaky, které jsou různých typů a které bude třeba nějakým způsobem převést do číselné reprezentace.
    
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroj dat

Budeme se zabývat predikcí přežití pasažérů Titaniku.
K dispozici máte trénovací data v souboru **data.csv** a data na vyhodnocení v souboru **evaluation.csv**.

#### Seznam příznaků:
* survived - zda přežil, 0 = Ne, 1 = Ano, **vysvětlovaná proměnná**, kterou chcete predikovat
* pclass - Třída lodního lístku, 1 = první, 2 = druhá, 3 = třetí
* name - jméno
* sex - pohlaví
* age - věk v letech
* sibsp	- počet sourozenců / manželů, manželek na palubě
* parch - počet rodičů / dětí na palubě
* ticket - číslo lodního lístku
* fare - cena lodního lístku
* cabin	- číslo kajuty
* embarked	- místo nalodění, C = Cherbourg, Q = Queenstown, S = Southampton
* home.dest - Bydliště/Cíl

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * V Jupyter notebooku načtěte data ze souboru **data.csv**. Vhodným způsobem si je rozdělte na trénovací, testovací a případně i validační množinu (preferujeme ale použití cross-validation).
  * Projděte si jednotlivé příznaky a transformujte je do vhodné podoby pro použití ve vybraném klasifikačním modelu.
  * Podle potřeby si můžete vytvářet nové příznaky (na základě existujících), například tedy můžete vytvořit příznak měřící délku jména. Některé příznaky můžete také úplně zahodit.
  * Nějakým způsobem se vypořádejte s chybějícími hodnotami.
  * Následně si vyberte vhodný klasifikační model z přednášek. Najděte vhodné hyperparametry a určete jeho přesnost (accuracy) na trénovací množině. Také určete jeho přesnost na testovací/vaidační množině.
  * Načtěte vyhodnocovací data ze souboru **evaluation.csv**. Napočítejte predikce pro tyto data (vysvětlovaná proměnná v nich již není). Vytvořte **results.csv** soubor, ve kterém tyto predikce uložíte do dvou sloupců: ID, predikce přežití. Tento soubor nahrajte do repozitáře.

**Další body zadání** za případné další body  (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +4 body) Aplikujte všechny klasifikační modely z přednášek a určete (na základě přesnosti na validační množině), který je nejlepší. Přesnost tohoto nejlepšího modelu odhadněte pomocí testovací množiny. K predikcím na vyhodnocovacích datech využijte tento model.
  * (až +4 body) Zkuste použít nějaké (alespoň dvě) netriviální metody doplňování chybějících hodnot u věku. Zaměřte na vliv těchto metod na přesnost predikce výsledného modelu. K predikcím na vyhodnocovacích datech využijte ten přístup, který Vám vyjde jako nejlepší.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) s predikcemi pro vyhodnocovací data.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. **První verze je ale důležitá a bude-li odbytá, budete za to penalizováni**

# Řešení

In [ ]:
import math
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
import sklearn.metrics as metrics

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Načtení dat

In [ ]:
data = pd.read_csv('data.csv')

# display(data.shape)
# display(data.head())
# display(data.info())
# display(data.describe())
# display(data.nunique())

## Úprava dat

### Dropnutí nevýznamných sloupců

In [ ]:
def drop_useless_data(df):
    return df.drop(["ID", "ticket", "home.dest"], axis=1)

data = drop_useless_data(data)

### Name -> Title

In [ ]:
def create_title_drop_name(df):
    title = [i.split(",")[1].split(".")[0].strip() for i in df["name"]]
    df["title"] = title
    return df.drop("name", axis=1)

data = create_title_drop_name(data)

### Fare

In [ ]:
def fill_fare(df):
    df["fare"] = df["fare"].fillna(df["fare"].median())
    return df

data = fill_fare(data)

### Cabin

In [ ]:
def adjust_cabin(df):
    df["cabin"] = df["cabin"].apply(lambda x: "X" if pd.isna(x) else x[:1])
    return df

data = adjust_cabin(data)

### Embarked

In [ ]:
def fill_embarked(df):
    df["embarked"] = df["embarked"].fillna("S")
    return df

data = fill_embarked(data)

### Převedení stringů na kategorie (category codes)

In [ ]:
def str_to_cat_codes(df):
    cat_columns = df.select_dtypes(['object']).columns
    df[cat_columns] = df[cat_columns].astype("category").apply(lambda x: x.cat.codes)
    return df

data = str_to_cat_codes(data)

### Age
#### Lineární regrese

In [ ]:
def fill_age_lr(df):    
    age_data = df.copy()
    age_data = age_data[age_data["age"].isna() == False]
    age_x_data = age_data.drop("age", axis=1)
    age_y_data = age_data["age"]
    missing_rows = df.copy()
    missing_rows = missing_rows[missing_rows["age"].isna()].drop("age", axis=1)

    lr = LinearRegression()
    lr.fit(age_x_data,age_y_data)
    
    missing_rows["age"] = lr.predict(missing_rows).round()
    return df[df["age"].isna() == False].append(missing_rows, sort=False)

data = fill_age_lr(data)

#### kNN

In [ ]:
def fill_age_knn(df):
    age_data = df.copy()
    age_data = age_data[age_data["age"].isna() == False]
    age_x_data = age_data.drop("age", axis=1)
    age_y_data = age_data["age"]
    missing_rows = df.copy()
    missing_rows = missing_rows[missing_rows["age"].isna()].drop("age", axis=1)

    knn = KNeighborsRegressor(n_neighbors=1, p=1, weights="uniform")
    knn.fit(age_x_data, age_y_data)
    
    missing_rows["age"] = knn.predict(missing_rows).round()
    return df[df["age"].isna() == False].append(missing_rows, sort=False)

# data = fill_age_knn(data)

## Modely predikce
### Rozdělení dat

In [ ]:
# separate data
Xdata = data.copy()
Xdata = Xdata.drop("survived", axis=1)
ydata = data.iloc[:,0]

# split data to train/validation/test
rd_seed = 1024
Xtrain, Xtest, ytrain, ytest = train_test_split(Xdata, ydata, test_size=0.25, random_state=rd_seed)
Xtrain, Xval, ytrain, yval = train_test_split(Xtrain, ytrain, test_size=0.25, random_state=rd_seed)

### Decision Tree Classifier

In [ ]:
def decision_tree_classifier(x_train, x_val, x_test, y_train, y_val, y_test):
    param_comb = ParameterGrid({
        'max_depth': range(1,30), 
        'criterion': ['entropy', 'gini']
    })

    val_acc = []
    train_acc = []
    for params in param_comb:
        dt = DecisionTreeClassifier(max_depth=params['max_depth'], criterion=params['criterion'])
        dt.fit(x_train, y_train)
        train_acc.append(metrics.accuracy_score(y_train, dt.predict(x_train)))
        val_acc.append(metrics.accuracy_score(y_val, dt.predict(x_val)))
    
    best_params = param_comb[np.argmax(val_acc)]
    dt = DecisionTreeClassifier(**best_params)
    dt.fit(x_train, y_train)
    score = metrics.accuracy_score(y_test, dt.predict(x_test))
    return ("Decision Tree Classifier", score, best_params)

dtc = decision_tree_classifier(Xtrain, Xval, Xtest, ytrain, yval, ytest)
dtc

### Random Forest Classifier

In [ ]:
def random_forest_classifier(x_train, x_val, x_test, y_train, y_val, y_test):
    param_comb = ParameterGrid({
        'max_depth': range(1,10), 
        'criterion': ['entropy', 'gini'],
        'n_estimators': range(1,50,5)
    })

    val_acc = []
    train_acc = []
    for params in param_comb:
        rf = RandomForestClassifier(max_depth=params['max_depth'],
                                    criterion=params['criterion'],
                                    n_estimators=params['n_estimators'])
        rf.fit(x_train, y_train)
        train_acc.append(metrics.accuracy_score(y_train, rf.predict(x_train)))
        val_acc.append(metrics.accuracy_score(y_val, rf.predict(x_val)))
    
    best_params = param_comb[np.argmax(val_acc)]
    rf = RandomForestClassifier(**best_params)
    rf.fit(x_train, y_train)
    score = metrics.accuracy_score(y_test, rf.predict(x_test))
    return ("Random Forest Classifier", score, best_params)

rfc = random_forest_classifier(Xtrain, Xval, Xtest, ytrain, yval, ytest)
rfc

### AdaBoost

In [ ]:
def ada_boost_classifier(x_train, x_val, x_test, y_train, y_val, y_test):
    param_comb = ParameterGrid({
        'learning_rate': [0.01, 0.05, 0.1, 0.3, 0.5, 1],
        'n_estimators': range(1,50,5)
    })

    val_acc = []
    train_acc = []
    for params in param_comb:
        ab = AdaBoostClassifier(learning_rate=params['learning_rate'], n_estimators=params['n_estimators'])
        ab.fit(x_train, y_train)
        train_acc.append(metrics.accuracy_score(y_train, ab.predict(x_train)))
        val_acc.append(metrics.accuracy_score(y_val, ab.predict(x_val)))
    
    best_params = param_comb[np.argmax(val_acc)]
    ab = AdaBoostClassifier(**best_params)
    ab.fit(x_train, y_train)
    score = metrics.accuracy_score(y_test, ab.predict(x_test))
    return ("AdaBoost Classifier", score, best_params)

abc = ada_boost_classifier(Xtrain, Xval, Xtest, ytrain, yval, ytest)
abc

### Výběr nejlepšího predikčního modelu

In [ ]:
models = [dtc, rfc, abc]

def best_model(models):
    best = None
    for model in models:
        if best is None:
            best = model
            continue
            
        name, score, params = model
        name_best, score_best, score_params = best
        
        if score > score_best:
            best = model
    return best

best_model(models)

## Predikce vyhodnocovacích dat

In [ ]:
eval_data = pd.read_csv("evaluation.csv")
eval_data = drop_useless_data(eval_data)
eval_data = create_title_drop_name(eval_data)
eval_data = fill_fare(eval_data)
eval_data = adjust_cabin(eval_data)
eval_data = str_to_cat_codes(eval_data)
eval_data = fill_age_lr(eval_data)

In [ ]:
dt = RandomForestClassifier(n_estimators=31, max_depth=6, criterion="entropy")
dt.fit(Xtrain, ytrain)
results = pd.DataFrame()
results["ID"] = range(0, eval_data.shape[0])
results["predikce přežití"] = dt.predict(eval_data)
results
# results[results["predikce přežití"] == 1].count()